In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-ml-challenge/sample_test.csv
/kaggle/input/amazon-ml-challenge/sample_test_out.csv
/kaggle/input/amazon-ml-challenge/evaluation.py
/kaggle/input/amazon-ml-challenge/train.csv
/kaggle/input/amazon-ml-challenge/test.csv
/kaggle/input/models/pytorch/models-by-shivansh/1/lgbm_regressor.pkl
/kaggle/input/models/pytorch/models-by-shivansh/1/price_processor.joblib
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/config.json
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/merges.txt
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/training_args.bin
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/vocab.json
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/tokenizer_config.json
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/model.safetensors
/kaggle/input/models/pytorch/models-by-shivansh/1/roberta_classifier/special_tokens_map.json
/kaggle/input/models

In [3]:
import scipy, transformers, accelerate, sklearn, joblib, re, os
from scipy import stats
from scipy.stats.mstats import winsorize
import lightgbm as lgb
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

In [4]:
import shutil
import os

SRC_MODEL_DIR = "/kaggle/input/models/pytorch/models-by-shivansh/1"
DST_MODEL_DIR = "/kaggle/working/models"

os.makedirs(DST_MODEL_DIR, exist_ok=True)

for item in os.listdir(SRC_MODEL_DIR):
    src = os.path.join(SRC_MODEL_DIR, item)
    dst = os.path.join(DST_MODEL_DIR, item)

    if os.path.isdir(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)
    else:
        shutil.copy2(src, dst)

print("✅ Models copied to writable directory")


✅ Models copied to writable directory


In [5]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 \
  --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 111.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 112.9 MB/s eta 0:00:0000:0100:01
     ━━━━

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.8.0+cu126
12.6
True
Tesla P100-PCIE-16GB


**Configuration**

In [7]:
# --- File Paths ---
DATA_DIR = "/kaggle/input/amazon-ml-challenge/"
TRAIN_FILE = f"{DATA_DIR}train.csv"
TEST_FILE = f"{DATA_DIR}test.csv"
MODEL_DIR = "/kaggle/working/models/"
SUBMISSION_FILE = "/kaggle/working/submission.csv"

# --- Preprocessing ---
PRICE_COLUMN = "price"
TEXT_COLUMN = "catalog_content"
TARGET_CLASS_COLUMN = "price_class"
NUM_PRICE_CLASSES = 14

# --- RoBERTa Model ---
ROBERTA_MODEL_NAME = "roberta-base"
ROBERTA_MODEL_PATH = f"{MODEL_DIR}roberta_classifier"
ROBERTA_BATCH_SIZE = 64      # Increased for speed
ROBERTA_EPOCHS = 2           # Reduced, often sufficient
ROBERTA_LEARNING_RATE = 2e-5

# --- Ensemble Model (LightGBM) ---
ENSEMBLE_MODEL_PATH = f"{MODEL_DIR}lgbm_regressor.pkl"
LGBM_PARAMS = {
    'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 2000,
    'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
    'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1,
    'num_leaves': 31, 'verbose': -1, 'n_jobs': -1,
}
SVM_PARAMS = {'kernel': 'rbf', 'C': 1.0, 'epsilon': 0.1}
DT_PARAMS = {'max_depth': 10, 'min_samples_split': 5}
ADABOOST_PARAMS = {'n_estimators': 50, 'learning_rate': 1.0}
VOTING_WEIGHTS = [0.3, 0.3, 0.4] # Adjust these based on your preference

# --- General ---
RANDOM_STATE = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

**Data PreProcessing**

In [8]:
# ----------------- Feature Engineering (from your provided code) -----------------
weight_units = {'g','gram','grams','gramm','gr','kg','k','lb','pound','pounds','oz','ounce','ounces'}
volume_units = {'ml','millilitre','milliliter','mililitro','ltr','liter','liters','l','fl oz','fluid ounce','fluid ounces','fluid ounces'}
count_units  = {'count','ct','each','unit','units','pack','packs','piece','pieces','box','bottle','bottles','bag','bags','case','carton','capsule','jar','pouch','bucket','tea bags','paper cupcake liners'}
length_units = {'cm','mm','m','meter','meters','inch','inches','in','ft','foot','feet'}

def extract_value_unit(text):
    lines = [l.strip() for l in str(text).split("\n") if l.strip()]
    if len(lines) < 2:
        return pd.Series([None, None])
    value_line = lines[-2]
    unit_line = lines[-1]
    match_val = re.search(r"[\d\.]+", value_line)
    value = float(match_val.group()) if match_val else None
    unit = re.sub(r'Unit[s]?:\s*', '', unit_line, flags=re.IGNORECASE).strip()
    return pd.Series([value, unit])

def extract_ipq(text):
    patterns = [r'pack of (\d+)', r'(\d+)\s*count', r'box of (\d+)', r'case of (\d+)', r'(\d+)\s*pieces', r'(\d+)\s*units', r'(\d+)\s*ct', r'(\d+)\s*each']
    text_lower = str(text).lower()
    for pat in patterns:
        match = re.search(pat, text_lower)
        if match:
            return int(match.group(1))
    return 1

def map_unit_class(unit):
    if unit is None: return 'others'
    u = unit.strip().lower()
    if u in weight_units: return 'weight'
    if u in volume_units: return 'volume'
    if u in count_units: return 'count'
    if u in length_units: return 'length'
    return 'others'

def convert_to_standard(value, unit):
    if value is None or unit is None: return value
    u = unit.strip().lower()
    if u in {'oz', 'ounce', 'ounces'}: return value * 28.3495
    if u in {'lb', 'pound', 'pounds'}: return value * 453.592
    if u in {'kg', 'k'}: return value * 1000
    if u in {'g', 'gram', 'grams', 'gramm', 'gr'}: return value
    if u in {'ltr', 'liter', 'liters', 'l'}: return value * 1000
    if u in {'fl oz', 'fluid ounce', 'fluid ounces'}: return value * 29.5735
    if u in {'ml', 'millilitre', 'milliliter', 'mililitro'}: return value
    if u in {'cm'}: return value / 100
    if u in {'mm'}: return value / 1000
    if u in {'inch', 'inches', 'in'}: return value * 0.0254
    if u in {'ft', 'foot', 'feet'}: return value * 0.3048
    if u in {'m', 'meter', 'meters'}: return value
    if u in count_units: return value
    return value

def engineer_features(df):
    """Applies all feature engineering steps."""
    temp_df = df.copy()
    temp_df[['value', 'unit']] = temp_df['catalog_content'].apply(extract_value_unit)
    temp_df['IPQ'] = temp_df['catalog_content'].apply(extract_ipq)
    temp_df['standardised_units'] = temp_df['unit'].apply(map_unit_class)
    temp_df['updated_values'] = temp_df.apply(lambda row: convert_to_standard(row['value'], row['unit']), axis=1)
    
    # Fill NaNs created during feature engineering
    temp_df['updated_values'] = temp_df['updated_values'].fillna(temp_df['updated_values'].median())
    temp_df['IPQ'] = temp_df['IPQ'].fillna(1)
    
    # One-hot encode and ensure all possible columns are present
    unit_dummies = pd.get_dummies(temp_df['standardised_units'], prefix='unit')
    for col in ['unit_weight', 'unit_volume', 'unit_count', 'unit_length', 'unit_others']:
        if col not in unit_dummies.columns:
            unit_dummies[col] = 0
    temp_df = pd.concat([temp_df, unit_dummies], axis=1)
    
    return temp_df

# ----------------- Price Transformation Pipeline -----------------

class PriceProcessor:
    """Handles Box-Cox, Winsorization, and Binning for the price column."""
    def __init__(self):
        self.lambda_ = None
        self.winsor_limits = (0.01, 0.01) # Winsorize 1% from both tails
        self.bin_edges = None
        self.path = os.path.join(MODEL_DIR, "price_processor.joblib")

    def fit_transform(self, price_series):
        # 1. Box-Cox Transform (add 1 to handle prices <= 0)
        transformed_price, self.lambda_ = stats.boxcox(price_series + 1)
        
        # 2. Winsorization
        winsorized_price = winsorize(transformed_price, limits=self.winsor_limits)
        
        # 3. Binning into classes
        binned_price, self.bin_edges = pd.qcut(
            winsorized_price, 
            q=NUM_PRICE_CLASSES, 
            labels=False, 
            retbins=True, 
            duplicates='drop'
        )
        self.save()
        return binned_price

    def save(self):
        os.makedirs(os.path.dirname(self.path), exist_ok=True)
        joblib.dump(self, self.path)

    @classmethod
    def load(cls):
        return joblib.load(os.path.join(MODEL_DIR, "price_processor.joblib"))

**Evaluation**

In [9]:
def smape(y_true, y_pred):
    # Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    
    # Handle the case where both true and pred are zero
    # to avoid division by zero
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    
    return np.mean(ratio) * 100

**Ensemble Model**

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor
from sklearn.impute import SimpleImputer

def get_feature_columns(df):
    """Identifies the feature columns for the ensemble model."""
    exclude_cols = [
        'sample_id', 'catalog_content', 'image_link', 'price',
        'price_class', 'value', 'unit', 'standardised_units'
    ]
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    # print(f"DEBUG: Selected {len(feature_cols)} numeric features: {feature_cols}")
    return feature_cols

def train_ensemble_model(df, target_col):
    """Trains and saves the Voting Regressor model (SVR + DecisionTree + AdaBoost)."""
    train_split, val_split = train_test_split(df, test_size=0.2, random_state=42)
    
    feature_cols = get_feature_columns(df)
    X_train = train_split[feature_cols]
    y_train = train_split[target_col]

    X_val = val_split[feature_cols]
    y_val = val_split[target_col]

    # Base learners
    svr = make_pipeline(StandardScaler(), SVR(**SVM_PARAMS))
    dtr = DecisionTreeRegressor(random_state=RANDOM_STATE, **DT_PARAMS)
    ada = AdaBoostRegressor(random_state=RANDOM_STATE, **ADABOOST_PARAMS)

    # Voting ensemble
    model = VotingRegressor(
        estimators=[
            ("svr", svr),
            ("dt", dtr),
            ("ada", ada),
        ],
        weights=VOTING_WEIGHTS,
        n_jobs=None,  # VotingRegressor in sklearn doesn't support n_jobs (only in some versions for estimators); keep None
    )

    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    train_predictions = model.predict(X_train)

    score_train = smape(y_train,train_predictions)
    score_val = smape(y_val, val_predictions)
    
    print(f"\n📉 Validation SMAPE Score: {score_val:.4f}%")
    print(f"\n📉 Training SMAPE Score: {score_train:.4f}%")
    print("----------------------------------------------\n")

    # Save the model
    os.makedirs(os.path.dirname(ENSEMBLE_MODEL_PATH), exist_ok=True)
    joblib.dump(model, ENSEMBLE_MODEL_PATH)
    print(f"Ensemble model saved to {ENSEMBLE_MODEL_PATH}")
    return model

def predict_with_ensemble_model(df):
    """Loads and predicts with the saved Voting Regressor model."""
    feature_cols = get_feature_columns(df)
    X_test = df[feature_cols]

    model = joblib.load(ENSEMBLE_MODEL_PATH)
    predictions = model.predict(X_test)
    return predictions


**Roberta Classifier**

In [11]:
class PriceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def train_roberta(train_df, text_col, target_col):
    """Fine-tunes and saves the RoBERTa model."""
    tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_MODEL_NAME)
    model = RobertaForSequenceClassification.from_pretrained(
        ROBERTA_MODEL_NAME,
        num_labels=train_df[target_col].nunique()
    ).to(DEVICE)

    train_dataset = PriceDataset(
        texts=train_df[text_col].to_numpy(),
        labels=train_df[target_col].to_numpy(),
        tokenizer=tokenizer
    )

    training_args = TrainingArguments(
        output_dir=ROBERTA_MODEL_PATH,
        num_train_epochs=ROBERTA_EPOCHS,
        per_device_train_batch_size=ROBERTA_BATCH_SIZE,
        learning_rate=ROBERTA_LEARNING_RATE,
        logging_dir='./logs',
        logging_steps=100,
        do_train=True,
        do_eval=False,
        save_strategy="epoch",
        fp16=True,  # Mixed-precision for speed
        load_best_model_at_end=False,
        seed=RANDOM_STATE,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model(ROBERTA_MODEL_PATH)
    tokenizer.save_pretrained(ROBERTA_MODEL_PATH)
    print(f"RoBERTa model saved to {ROBERTA_MODEL_PATH}")

import gc

def predict_with_roberta(df, text_col, chunk_size=5000):
    """
    Optimized prediction:
    1. Uses FP16 and Batch Size 64 (Speed).
    2. Processes in chunks to prevent OOM (Memory).
    3. Clears cache aggressively.
    """
    print(f"🔄 Initializing Inference on {len(df)} rows...")
    
    tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_MODEL_PATH)
    model = RobertaForSequenceClassification.from_pretrained(ROBERTA_MODEL_PATH).to(DEVICE)
    
    # optimized arguments for inference
    test_args = TrainingArguments(
        output_dir=os.path.join(MODEL_DIR, "inference_temp"),
        per_device_eval_batch_size=64,   # Match your training batch size
        fp16=True,                       # Much faster, less memory
        dataloader_num_workers=2,        # Speed up data loading
        report_to="none",
    )
    
    trainer = Trainer(model=model, args=test_args)
    
    all_predictions = []
    
    # Process in chunks to avoid RAM explosion
    total_chunks = (len(df) // chunk_size) + 1
    
    for i in range(0, len(df), chunk_size):
        chunk_df = df.iloc[i : i + chunk_size]
        
        if chunk_df.empty:
            continue
            
        print(f"   Processing chunk {i//chunk_size + 1}/{total_chunks} ({len(chunk_df)} rows)...")
        
        # Create dataset just for this chunk
        chunk_dataset = PriceDataset(
            texts=chunk_df[text_col].to_numpy(),
            labels=np.zeros(len(chunk_df)), # Dummy labels
            tokenizer=tokenizer
        )
        
        # Predict
        predictions = trainer.predict(chunk_dataset).predictions
        
        # Get class indices and append to list
        chunk_preds = np.argmax(predictions, axis=1)
        all_predictions.append(chunk_preds)
        
        # CRITICAL: Free memory
        del chunk_dataset, predictions, chunk_preds
        torch.cuda.empty_cache()
        gc.collect()
        
    print("✅ Inference complete.")
    return np.concatenate(all_predictions)

In [12]:
def get_roberta_predictions_with_cache(
    df,
    text_col,
    cache_name,
):
    """
    Computes RoBERTa predictions once and caches them.
    On reruns, loads directly from disk.
    """
    CACHE_DIR = "/kaggle/working/roberta_cache"
    os.makedirs(CACHE_DIR, exist_ok=True)

    cache_path = os.path.join(CACHE_DIR, f"{cache_name}.npy")

    if os.path.exists(cache_path):
        print(f"✅ Loading cached RoBERTa predictions from {cache_path}")
        return np.load(cache_path)

    print(f"🚀 Running RoBERTa inference for {cache_name} (this may take time)...")
    preds = predict_with_roberta(df, text_col)

    np.save(cache_path, preds)
    print(f"💾 Saved RoBERTa predictions to {cache_path}")

    return preds


**Main**

In [13]:
def main():
    #--- 0. CRITICAL: Verify GPU is available ---
    if not torch.cuda.is_available():
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! ERROR: No CUDA-enabled GPU found !!!")
        print("!!!       Aborting training.       !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return
    print(f"PyTorch confirmed CUDA is available. Using device: {DEVICE.upper()}")
    
    # --- 1. Load Data ---
    print("\nStep 1: Loading data...")
    train_df = pd.read_csv(TRAIN_FILE)
    test_df = pd.read_csv(TEST_FILE)
    print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")

    # --- 2. Preprocess Price Column for Training ---
    print("\nStep 2: Preprocessing price column for RoBERTa training...")
    price_processor = PriceProcessor()
    train_df[TARGET_CLASS_COLUMN] = price_processor.fit_transform(train_df[PRICE_COLUMN])
    print(f"Price column transformed and binned into {train_df[TARGET_CLASS_COLUMN].nunique()} classes.")

    # --- 3. Fine-Tune RoBERTa Classifier ---
    print("\nStep 3: Fine-tuning RoBERTa for price classification...")
    model_config_path = os.path.join(ROBERTA_MODEL_PATH, "config.json")
    if not os.path.exists(model_config_path):
        print("RoBERTa model not found. Starting training...")
        train_roberta(train_df, TEXT_COLUMN, TARGET_CLASS_COLUMN)
    else:
        print("Found existing RoBERTa model. Skipping training.")

    # --- 4. Get RoBERTa Predictions as Features ---
    print("\nStep 4: Generating price class predictions from RoBERTa (with caching)...")

    train_df['roberta_pred_class'] = predict_with_roberta(train_df, TEXT_COLUMN)
    test_df['roberta_pred_class'] = predict_with_roberta(test_df, TEXT_COLUMN)

    print("✅ RoBERTa predictions added as features.")

    # --- 5. Engineer Additional Features ---
    print("\nStep 5: Engineering features from catalog_content...")
    train_df_featured = engineer_features(train_df)
    test_df_featured = engineer_features(test_df)
    print("Feature engineering complete.")

    # --- 6. Train Ensemble Model ---
    print("\nStep 6: Training the final LightGBM ensemble model...")
    train_ensemble_model(train_df_featured, PRICE_COLUMN)

    # flag = input("Do you want to use this model for test predictions?(Y/N) : ")
    # if(flag in 'Nn'):
    #     return
    return
    # --- 7. Make Final Price Predictions ---
    print("\nStep 7: Making final price predictions on the test set...")
    test_predictions = predict_with_ensemble_model(test_df_featured)
    test_df['price'] = test_predictions

    # --- 8. Save Results ---
    # The evaluation step has been removed as test.csv does not have a 'price' column.
    print("\nStep 8: Saving submission file...")
    submission_df = test_df[['sample_id', 'price']]
    submission_df.to_csv(SUBMISSION_FILE, index=False)
    print(f"✅ Submission file saved to {SUBMISSION_FILE}")

main()

PyTorch confirmed CUDA is available. Using device: CUDA

Step 1: Loading data...
Train shape: (75000, 4), Test shape: (75000, 3)

Step 2: Preprocessing price column for RoBERTa training...


/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4968: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


Price column transformed and binned into 14 classes.

Step 3: Fine-tuning RoBERTa for price classification...
Found existing RoBERTa model. Skipping training.

Step 4: Generating price class predictions from RoBERTa (with caching)...
🔄 Initializing Inference on 75000 rows...
   Processing chunk 1/16 (5000 rows)...


   Processing chunk 2/16 (5000 rows)...


   Processing chunk 3/16 (5000 rows)...


...

   Processing chunk 12/16 (5000 rows)...


   Processing chunk 13/16 (5000 rows)...


   Processing chunk 14/16 (5000 rows)...


   Processing chunk 15/16 (5000 rows)...


✅ Inference complete.
✅ RoBERTa predictions added as features.

Step 5: Engineering features from catalog_content...
Feature engineering complete.

Step 6: Training the final LightGBM ensemble model...

📉 Validation SMAPE Score: 54.2641%

📉 Training SMAPE Score: 52.7590%
----------------------------------------------

Ensemble model saved to /kaggle/working/models/lgbm_regressor.pkl
